In [1]:
import pickle
import pandas as pd
import os           
import numpy as np
import re
import itertools
import plotly.express as px
#import brightway2 as bw 
import random 
import string

In [2]:
import sys
sys.path.append('../../Module')  #two levels up & then down to Module folder
#from recipe_mod import *
from recipe_mod_resIC import *

<h3> raw RIVM and BW2 seperate land use to land transformation and land occupation  </h3>
<li> land transformation   </li>
<li> land occupation </li>
<font color = 'red'> read in seperately then combine to land use, as SP and oLCA have final land use (incl. LT and LO) </font>

In [3]:
lcia_name = "Land use"
SP_name = "Land use"
olca_filepathname = "../../Data_rawCFs/olca_recipe_LU_saved.dat"
rivm_name_LT = "Land transformation"
rivm_name_LO = "Land occupation"
bw2_filepathname_LT = "../../Data_rawCFs/BW2_recipe_Land transformation.dat"
bw2_filepathname_LO = "../../Data_rawCFs/BW2_recipe_Land occupation.dat" 

# used in final calculation to create dict names
pairwise_name = ["SPvsRIVM", "SPvsBW2", "SPvsopenLCA", "openLCAvsBW2", "openLCAvsRIVM", "BW2vsRIVM"]

### read in SP LCIA

In [4]:
SP_recipe = pd.read_excel("../../Data_rawCFs/SP_ReCiPe.xlsx", sheet_name=SP_name, 
                                  header=[0], converters={'CAS':str})

print(len(SP_recipe))
SP_recipe.fillna(''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(9)), inplace=True)
#to assign a random string_num to CAS with NA, so that they're not the "SAME" CAS when compared to other sources:
#print ([x for x in SP_recipe["CAS"] if '-' not in x ] [0]) 
SP_recipe.columns

91


Index(['Compartment', 'Subcompartment', 'Substance', 'CAS', 'Factor', 'Unit'], dtype='object')

In [5]:
SP_recipe = SP_recipe.rename(columns={"Compartment": "category", "Subcompartment": "subcategory", 
                               "Substance": "flow", "Factor":"value", "CAS": "cas_number" })
SP_recipe = SP_recipe.rename(str.lower, axis='columns')
SP_recipe['flow'] = SP_recipe["flow"].str.lower()
SP_recipe.head()

,category,subcategory,flow,cas_number,value,unit
0,Raw,(unspecified),"occupation, agriculture",QC4CGTTL5,1.0,m2a crop eq / m2a
1,Raw,(unspecified),"occupation, agriculture, mosaic",QC4CGTTL5,1.0,m2a crop eq / m2a
2,Raw,(unspecified),"occupation, annual crop",QC4CGTTL5,1.0,m2a crop eq / m2a
3,Raw,(unspecified),"occupation, annual crop, conservation tillage",QC4CGTTL5,1.0,m2a crop eq / m2a
4,Raw,(unspecified),"occupation, annual crop, conventional tillage",QC4CGTTL5,1.0,m2a crop eq / m2a


### read in openlca LCIA

In [6]:
olca_recipe = unpickle_data(olca_filepathname)
olca_recipe.fillna(''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(9)), inplace=True)
#print ([x for x in olca_recipe["CAS_NUMBER"] if '-' not in x ] [0]) 
print(len(olca_recipe))
olca_recipe = olca_recipe.rename(str.lower, axis='columns')

160


In [7]:
olca_recipe['flow'] = olca_recipe["flow"].str.lower()
olca_recipe.head()

,impact_method,impact_category_uuid,impact_category,reference_unit,flow_uuid,flow,cas_number,subcategory,category,unit,value
0,ReCiPe 2016 Midpoint (H),f7c097c5-71c7-322c-a610-5dee0a39fd6b,Land use,m2a crop eq,70c4c6d8-ed81-4763-ae6d-39e54ef0b1fa,"occupation, seabed, drilling and mining",GO7V5LUOO,land,Resource,m2*a,0.0
1,ReCiPe 2016 Midpoint (H),f7c097c5-71c7-322c-a610-5dee0a39fd6b,Land use,m2a crop eq,9db06277-b6d9-4c48-8cfb-de342e928a50,"occupation, seabed, infrastructure",GO7V5LUOO,land,Resource,m2*a,0.0
2,ReCiPe 2016 Midpoint (H),f7c097c5-71c7-322c-a610-5dee0a39fd6b,Land use,m2a crop eq,d903114a-8c2c-3649-ab5b-22f404fc4646,"occupation, annual crop, reduced tillage",GO7V5LUOO,land,Resource,m2*a,1.0
3,ReCiPe 2016 Midpoint (H),f7c097c5-71c7-322c-a610-5dee0a39fd6b,Land use,m2a crop eq,ded8e6ec-41f4-3417-8fee-a44d8a35bd77,"occupation, annual crop, conventional tillage",GO7V5LUOO,land,Resource,m2*a,1.0
4,ReCiPe 2016 Midpoint (H),f7c097c5-71c7-322c-a610-5dee0a39fd6b,Land use,m2a crop eq,dbc67ab5-f891-33ff-b1f2-eed940a1205b,"occupation, annual crop, conservation tillage",GO7V5LUOO,land,Resource,m2*a,1.0


### read in BW2 LCIA_LT and LO

In [8]:
bw_recipe_LT = unpickle_data(bw2_filepathname_LT)
bw_recipe_LT.fillna(''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(9)), inplace=True)
print(len(bw_recipe_LT))
bw_recipe_LT['flow'] = bw_recipe_LT["flow"].str.lower()
bw_recipe_LT.head()

9


,cas_number,flow,flow_uuid,type,category,subcategory,unit,value
0,D1RVQHAAY,"transformation, from grassland, natural (non-use)",b905c2e0-a0db-4e66-80d2-8bdfc93c6218,natural resource,natural resource,land,square meter,3.75
1,D1RVQHAAY,"transformation, from forest, primary (non-use)",9d136389-8670-45ae-ad1d-a06848a1fd1d,natural resource,natural resource,land,square meter,36.75
2,D1RVQHAAY,"transformation, from forest, secondary (non-use)",9cc80820-fcf9-4ea9-8e67-0aa423eba6a4,natural resource,natural resource,land,square meter,36.75
3,D1RVQHAAY,"transformation, from shrub land, sclerophyllous",17a5a406-333f-4b9e-8852-c2de50bc9585,natural resource,natural resource,land,square meter,3.75
4,D1RVQHAAY,"transformation, from wetland, inland (non-use)",0782d8ff-80e5-47a7-a2ba-3ba40ab60b60,natural resource,natural resource,land,square meter,3.75


In [9]:
bw_recipe_LO = unpickle_data(bw2_filepathname_LO)
bw_recipe_LO.fillna(''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(9)), inplace=True)
print(len(bw_recipe_LO))
bw_recipe_LO['flow'] = bw_recipe_LO["flow"].str.lower()
bw_recipe_LO.head()

41


,cas_number,flow,flow_uuid,type,category,subcategory,unit,value
0,5FNTYPKRJ,"occupation, pasture, man made, intensive",98b723d4-3362-4b38-8b1e-1bedacfd5e27,natural resource,natural resource,land,square meter-year,0.55
1,5FNTYPKRJ,"occupation, permanent crop, non-irrigated, int...",c9461a73-d00a-4fc7-a890-a9eda6af3185,natural resource,natural resource,land,square meter-year,0.70
2,5FNTYPKRJ,"occupation, mineral extraction site",379ba5c9-5c3a-43d0-8e2d-605ad9c39e46,natural resource,natural resource,land,square meter-year,0.73
3,5FNTYPKRJ,"occupation, annual crop, greenhouse",9e80f7cd-47fa-4c7f-8f2c-bdb9731b3196,natural resource,natural resource,land,square meter-year,0.73
4,5FNTYPKRJ,"occupation, permanent crop, irrigated, intensive",3afbc2d8-1b51-4d43-87a7-e68291c1e999,natural resource,natural resource,land,square meter-year,0.70


<font color = 'red'> combine LT and LO to one dataframe  </font>

In [10]:
bw_recipe = pd.concat([bw_recipe_LO, bw_recipe_LT], axis=0, ignore_index=True)

### read in raw RIVM_ReCiPe_LT and LO

In [11]:
rivm_recipe_LT = pd.read_excel("../../Data_rawCFs/ReCiPe2016_CFs_v1.1_20180117.xlsx", sheet_name=rivm_name_LT, 
                                  header=[1], converters={'CAS':str})
rivm_recipe_LT.head()

,Name in ecoinvent,Unnamed: 1,Individualist,Hierarchist,Egalitarian
0,"Transformation, from grassland, natural (non-use)",NaN,3.75,3.75,3.75
1,"Transformation, from forest, primary (non-use)",NaN,36.75,36.75,36.75
2,"Transformation, from forest, secondary (non-use)",NaN,36.75,36.75,36.75
3,"Transformation, from shrub land, sclerophyllous",NaN,3.75,3.75,3.75
4,"Transformation, from wetland, inland (non-use)",NaN,3.75,3.75,3.75


In [12]:
rivm_recipe_LT = rivm_recipe_LT.rename(columns={"Name in ecoinvent": "flow", "Hierarchist":"value"})
rivm_recipe_LT = rivm_recipe_LT.rename(str.lower, axis='columns') 
rivm_recipe_LT['flow'] = rivm_recipe_LT["flow"].str.lower()

In [13]:
rivm_recipe_LO = pd.read_excel("../../Data_rawCFs/ReCiPe2016_CFs_v1.1_20180117.xlsx", sheet_name=rivm_name_LO, 
                                  header=[1], converters={'CAS':str})
rivm_recipe_LO.head()

,Name in ecoinvent,Name in ReCiPe,Individualist,Hierarchist,Egalitarian,Unnamed: 5,Unnamed: 6,Crop type,CFocc (annual crop eq)
0,"Occupation, pasture, man made, intensive",pasture,0.55,0.55,0.55,NaN,NaN,managed forest,0.30
1,"Occupation, permanent crop, non-irrigated, int...",permanent crops,0.70,0.70,0.70,NaN,NaN,annual crops,1.00
2,"Occupation, mineral extraction site",artificial area,0.73,0.73,0.73,NaN,NaN,pasture,0.55
3,"Occupation, annual crop, greenhouse",artificial area,0.73,0.73,0.73,NaN,NaN,artificial area,0.73
4,"Occupation, permanent crop, irrigated, intensive",permanent crops,0.70,0.70,0.70,NaN,NaN,permanent crops,0.70


In [14]:
rivm_recipe_LO = rivm_recipe_LO.rename(columns={"Name in ecoinvent": "flow", "Hierarchist":"value"})
rivm_recipe_LO = rivm_recipe_LO.rename(str.lower, axis='columns') 
rivm_recipe_LO['flow'] = rivm_recipe_LO["flow"].str.lower()

In [15]:
rivm_recipe = pd.concat([rivm_recipe_LO, rivm_recipe_LT], axis=0, ignore_index=True)

### 3. final calculation, pair-wise comparison, corr. matrix

In [16]:
# create 6 empty dict: dict_SPvsRIVM; dict_SPvsBW2; dict_SPvsopenLCA; dict_openLCAvsBW2; dict_openLCAvsRIVM; dict_BW2vsRIVM
# each dict with five keys and null values (to be assigned later)
for name in pairwise_name:
    #print(name)
    keys = [name+"_commonEF_sumtable", name+"_commonEF_list", name + "_result_diffEFs", name + "_result_corr" ]
    globals()['dict_%s' % name] = dict.fromkeys(keys, None)

In [17]:
k_list = list(dict_SPvsRIVM)
dict_SPvsRIVM[k_list[0]], dict_SPvsRIVM[k_list[1]] = res_compare_included_EF(SP_recipe, "flow", rivm_recipe,  "flow", to_print = "no" )
if len(dict_SPvsRIVM[k_list[1]] ) == 0: 
    dict_SPvsRIVM[k_list[2]] = pd.DataFrame ({'SP_value': np.NaN , 'RIVM_value': np.NaN }, index=[0])
else:
    dict_SPvsRIVM[k_list[2]] = pd.DataFrame (
        {'SP_value': final_EF_combined(SP_recipe, "flow", "value", rivm_recipe, "flow", "value", dict_SPvsRIVM[k_list[1]])[0] , 
         'RIVM_value': final_EF_combined(SP_recipe, "flow", "value", rivm_recipe, "flow", "value", dict_SPvsRIVM[k_list[1]])[1] }, 
        index = dict_SPvsRIVM[k_list[1]]
    )

Detailed differences won't be printed out, to print, add input argument: to_print = 'yes'.  
N of common EFs to be used is: 37


In [18]:
k_list = list(dict_SPvsBW2)
dict_SPvsBW2[k_list[0]],dict_SPvsBW2[k_list[1]] = res_compare_included_EF(SP_recipe, "flow", bw_recipe,  "flow", to_print = "no" )
if len(dict_SPvsBW2[k_list[1]] ) == 0: 
    dict_SPvsBW2[k_list[2]] = pd.DataFrame ({'SP_value': np.NaN , 'BW2_value': np.NaN }, index=[0])
else:
    dict_SPvsBW2[k_list[2]] = pd.DataFrame (
        {'SP_value': final_EF_combined(SP_recipe, "flow", "value", bw_recipe, "flow", "value", dict_SPvsBW2[k_list[1]])[0] , 
         'BW2_value': final_EF_combined(SP_recipe, "flow", "value", bw_recipe, "flow", "value", dict_SPvsBW2[k_list[1]])[1] }, 
        index = dict_SPvsBW2[k_list[1]]
    )

Detailed differences won't be printed out, to print, add input argument: to_print = 'yes'.  
N of common EFs to be used is: 47


In [19]:
k_list = list(dict_SPvsopenLCA)
dict_SPvsopenLCA[k_list[0]],dict_SPvsopenLCA[k_list[1]] = res_compare_included_EF(SP_recipe, "flow", olca_recipe,  "flow", to_print = "no" )
if len(dict_SPvsopenLCA[k_list[1]] ) == 0: 
    dict_SPvsopenLCA[k_list[2]] = pd.DataFrame ({'SP_value': np.NaN , 'olca_value': np.NaN }, index=[0])
else:
    dict_SPvsopenLCA[k_list[2]] = pd.DataFrame (
        {'SP_value': final_EF_combined(SP_recipe, "flow", "value", olca_recipe, "flow", "value", dict_SPvsopenLCA[k_list[1]])[0] , 
         'olca_value': final_EF_combined(SP_recipe, "flow", "value", olca_recipe, "flow", "value", dict_SPvsopenLCA[k_list[1]])[1] },
        index = dict_SPvsopenLCA[k_list[1]]
    )

Detailed differences won't be printed out, to print, add input argument: to_print = 'yes'.  
N of common EFs to be used is: 91


In [20]:
k_list = list(dict_openLCAvsBW2)
dict_openLCAvsBW2[k_list[0]],dict_openLCAvsBW2[k_list[1]] = res_compare_included_EF(olca_recipe, "flow_uuid", bw_recipe, "flow_uuid", to_print = "no" )

if len(dict_openLCAvsBW2[k_list[1]] ) == 0: 
    dict_openLCAvsBW2[k_list[2]] = pd.DataFrame ({'olca_value': np.NaN , 'BW2_value': np.NaN }, index=[0])
else:
    dict_openLCAvsBW2[k_list[2]] = pd.DataFrame (
        {'olca_value': final_EF_combined(olca_recipe, "flow_uuid", "value", bw_recipe, "flow_uuid", "value", 
                                       dict_openLCAvsBW2[k_list[1]])[0] , 
         'BW2_value': final_EF_combined(olca_recipe, "flow_uuid", "value", bw_recipe, "flow_uuid", "value",
                                         dict_openLCAvsBW2[k_list[1]])[1] },
        index =  dict_openLCAvsBW2[k_list[1]]
    )

Detailed differences won't be printed out, to print, add input argument: to_print = 'yes'.  
N of common EFs to be used is: 47


In [21]:
k_list = list(dict_openLCAvsRIVM)
dict_openLCAvsRIVM[k_list[0]],dict_openLCAvsRIVM[k_list[1]] = res_compare_included_EF(olca_recipe, "flow", rivm_recipe, "flow", to_print = "no" )

if len(dict_openLCAvsRIVM[k_list[1]] ) == 0: 
    dict_openLCAvsRIVM[k_list[2]] = pd.DataFrame ({'olca_value': np.NaN , 'RIVM_value': np.NaN }, index=[0])
else:
    dict_openLCAvsRIVM[k_list[2]] = pd.DataFrame (
        {'olca_value': final_EF_combined(olca_recipe, "flow", "value", rivm_recipe, "flow", "value", 
                                       dict_openLCAvsRIVM[k_list[1]])[0] , 
         'BW2_value': final_EF_combined(olca_recipe, "flow", "value", rivm_recipe, "flow", "value",
                                         dict_openLCAvsRIVM[k_list[1]])[1] },
        index = dict_openLCAvsRIVM[k_list[1]]
    )

Detailed differences won't be printed out, to print, add input argument: to_print = 'yes'.  
N of common EFs to be used is: 38


In [22]:
k_list = list(dict_BW2vsRIVM)
dict_BW2vsRIVM[k_list[0]],dict_BW2vsRIVM[k_list[1]] = res_compare_included_EF(bw_recipe, "flow", rivm_recipe, "flow", to_print = "no" )
if len(dict_BW2vsRIVM[k_list[1]] ) == 0: 
    dict_BW2vsRIVM[k_list[2]] = pd.DataFrame ({'BW2_value': np.NaN , 'RIVM_value': np.NaN }, index=[0])
else:
    dict_BW2vsRIVM[k_list[2]] = pd.DataFrame (
        {'BW2_value': final_EF_combined(bw_recipe, "flow", "value", rivm_recipe, "flow", "value", 
                                       dict_BW2vsRIVM[k_list[1]])[0] , 
         'RIVM_value': final_EF_combined(bw_recipe, "flow", "value", rivm_recipe, "flow", "value",
                                         dict_BW2vsRIVM[k_list[1]])[1] },
        index = dict_BW2vsRIVM[k_list[1]]
    )

Detailed differences won't be printed out, to print, add input argument: to_print = 'yes'.  
N of common EFs to be used is: 39


In [23]:
for name in pairwise_name:
    new_df = globals()['dict_%s' % name][name + "_result_diffEFs"]
    try:
        new_df_corr = new_df.corr()
    except: 
        error
    print(new_df_corr)
    globals()['dict_%s' % name][name + "_result_corr"] = new_df_corr

            SP_value  RIVM_value
SP_value         1.0         1.0
RIVM_value       1.0         1.0
           SP_value  BW2_value
SP_value   1.000000   0.999953
BW2_value  0.999953   1.000000
            SP_value  olca_value
SP_value         1.0         1.0
olca_value       1.0         1.0
            olca_value  BW2_value
olca_value    1.000000   0.999953
BW2_value     0.999953   1.000000
            olca_value  BW2_value
olca_value         1.0        1.0
BW2_value          1.0        1.0
            BW2_value  RIVM_value
BW2_value         1.0         1.0
RIVM_value        1.0         1.0


In [24]:
# saving each dict_ to a single worksheet, different rows , # saving to sub-folder "results/Resources_IC/"

if not os.path.exists("results/Resources_IC"):
    os.makedirs("results/Resources_IC")

filename = lcia_name + "_pairwise_comp_result.xlsx"
writer = pd.ExcelWriter("results/Resources_IC/" + filename, engine='xlsxwriter')

for name in pairwise_name:
    sum_table = globals()['dict_%s' % name][name + "_commonEF_sumtable"]
    new_df = globals()['dict_%s' % name][name + "_result_diffEFs"]
    new_df_corr = globals()['dict_%s' % name][name + "_result_corr"]

    sum_table.to_excel(writer,sheet_name=name,startrow=0 , startcol=0)   # write first  (3rows)
    new_df_corr.to_excel(writer,sheet_name=name,startrow=5 , startcol=0) # write next   (3rows)
    new_df.to_excel(writer,sheet_name=name,startrow=10 , startcol=0)     # finally, write the resulting EFs 
     
writer.save()